Quick overview of `windowed_stateful_pass` - this probably should go to docs once API is stabilized!

In [1]:
import os
import sys

# PYTHONPATH shenanigans - to allow importing fastats
current_dir = os.path.abspath(os.curdir)
sys.path.append(os.path.abspath(current_dir + "/../../.."))

import fastats

In [2]:
from fastats import fs
from fastats.core.windowed_pass import windowed_pass, windowed_stateful_pass

import numpy as np
import pandas as pd

In [3]:
input_arr = np.arange(10e6, dtype='float')


@fs
def plain_sum(x):
    return np.sum(x)

In [4]:
pd_series = pd.Series(input_arr)

%timeit pd_series.rolling(10).sum()

811 ms ± 32.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
# this will re-compile on each run
%timeit windowed_pass(input_arr, 10, value=plain_sum)

703 ms ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Even with the compile overhead, it still outperforms pandas. Let's see how it will perform in pre-compiled mode (`return_callable`).

In [6]:
# this compiles the rolling fuction only once
callable_pass = windowed_pass(value=plain_sum, return_callable=True)
callable_pass(input_arr, 10)

%timeit callable_pass(input_arr, 10)

474 ms ± 396 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


Now, let's see how fast we can get it with `windowed_stateful_pass`.

In [7]:
@fs
def rolling_sum(x, val_in, val_out, state):
    if state.size == 0:   # state is empty
        state = np.array([np.sum(x)], dtype=x.dtype)
    else:
        state[0] += val_in - val_out
    return state[0], state

In [8]:
%timeit windowed_stateful_pass(input_arr, 10, value=rolling_sum)

527 ms ± 11.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
callable_stateful_pass = windowed_stateful_pass(value=rolling_sum, return_callable=True)
callable_stateful_pass(input_arr, 10)

%timeit callable_stateful_pass(input_arr, 10)

99.9 ms ± 2.65 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Quite impressive! The "stateful" version avoids re-calculating the full sum in each iteration for a significant speedup.

In [10]:
print("pandas")
%timeit pd_series.rolling(100).sum()

print("\nfastats windowed_pass")
%timeit callable_pass(input_arr, 100)

print("\nfastats windowed_stateful_pass")
%timeit callable_stateful_pass(input_arr, 100)

pandas
794 ms ± 37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

fastats windowed_pass
1.79 s ± 1.15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

fastats windowed_stateful_pass
97.9 ms ± 513 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


We can see that the re-summing overhead for large window values is significant ($O(n * win)$ complexity). `windowed_pass` is therefore slower than pandas here. However, pandas and `windowed_stateful_pass` maintain the same run time, the window size doesn't matter.

In [11]:
# the same run time for window size 10, 100 etc
%timeit callable_stateful_pass(input_arr, 1000)

98.7 ms ± 661 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Lastly, let's compare fastats against pure numba rolling sum function.

In [12]:
from numba import jit


@jit(nopython=True)
def windowed_stateful_pass_numba(x, win):
    result = np.full_like(x, np.nan)
    state = np.sum(x[0:win])
    
    result[win-1] = state

    for i in range(win+1, x.shape[0]+1):
        start = i - win
        in_idx = i - 1

        val_in = x[in_idx]
        val_out = x[start-1]
        state += val_in - val_out
        result[in_idx] = state

    return result

In [13]:
from numpy.testing import assert_allclose


assert_allclose(windowed_stateful_pass_numba(input_arr, 100),
                callable_stateful_pass(input_arr, 100))

print("Pure numba")
%timeit windowed_stateful_pass_numba(input_arr, 100)

print("\nfastats")
%timeit callable_stateful_pass(input_arr, 100)

Pure numba
73.6 ms ± 2.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

fastats
101 ms ± 1.84 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


This shows that there's some overhead of function calling and argument passing, but fastats window pass functions can run close to native numba speed, easily outperforming pandas.

# Open problems:
- Generic NaN handling in windows